In [5]:
import pandas as pd
import numpy as np 
import polars as pl 
import folium
import requests
import keplergl as kp
import geopandas as gpd
from gtfs_functions import Feed
import openrouteservice as ors

#https://santiagoa-toso.medium.com/python-for-gtfs-line-frequencies-in-a-map-d1c967695e3d
#https://towardsdatascience.com/python-for-gtfs-segment-frequencies-in-a-map-4dc3bc1e26ff
# https://medium.com/@mladen.dragicevic/real-time-gtfs-data-dashboard-in-python-209801ba32f1

In [2]:
def read_file(file):
    path = 'transit_cleaned/'

    df = pd.read_csv(path + file + '.txt')
    df.columns = df.columns.str.strip()
    
    return df

In [5]:
#calendar = read_file('calendar')
#calendar_dates = read_file('calendar_dates')
#directions = read_file('directions')
routes = read_file('routes')
shapes = read_file('shapes')
stops = read_file('stops')
stop_times = read_file('stop_times')
trips = read_file('trips')

#calendar = calendar.drop(columns='Unnamed: 0')
#calendar_dates = calendar_dates.drop(columns='Unnamed: 0')
#directions = directions.drop(columns='')
# routes = routes.drop(columns=['', 'route_desc', 'agency_id', 'route_url'])
# shapes = shapes.drop(columns='Unnamed: 0')
# stops = stops.drop(columns=['', 'stop_desc', 'stop_url'])
# #stop_times = stop_times.drop(columns=['Unnamed: 0', 'stop_headsign'])
# trips = trips.drop(columns=['', 'trip_short_name'])

# routes_trips = pd.merge(trips.astype('string'), routes.astype('string'), on='route_id', how='left')
# shapes_with_routes = pd.merge(shapes.astype('string'), routes_trips[['shape_id', 'route_short_name', 'route_type']], on='shape_id', how='left')
# shapes_with_routes = shapes_with_routes.drop_duplicates()
# shapes_with_routes[['shape_id', 'shape_pt_sequence']] = shapes_with_routes[['shape_id', 'shape_pt_sequence']].astype('int64')
# shapes_with_routes[['shape_pt_lat', 'shape_pt_lon', 'shape_dist_traveled']] = shapes_with_routes[['shape_pt_lat', 'shape_pt_lon', 'shape_dist_traveled']].astype('float64')

In [7]:
feed = Feed("transit_cleaned.zip")

In [38]:
# path = 'transit_cleaned/'

# shapes = read_file('shapes')
# shapes = shapes.dropna()

# df.to_csv(path + file + '.txt')



In [9]:
feed_routes = feed.routes
feed_trips = feed.trips
feed_stops = feed.stops
feed_stop_times = feed.stop_times
feed_shapes = feed.shapes
feed_lines_freq = feed.lines_freq

INFO:root:Reading "routes.txt".
INFO:root:accessing trips
INFO:root:Start date is None. You should either specify a start date or set busiest_date to True.
INFO:root:Reading "trips.txt".
INFO:root:Reading "calendar.txt".
INFO:root:Reading "calendar_dates.txt".
INFO:root:The busiest date/s of this feed or your selected date range is/are:  ['2024-12-26'] with 43046 trips.
INFO:root:In the case that more than one busiest date was found, the first one will be considered.
INFO:root:In this case is 2024-12-26.
INFO:root:Reading "stop_times.txt".
INFO:root:_trips is defined in stop_times
INFO:root:Reading "stops.txt".
INFO:root:computing patterns
INFO:root:Reading "shapes.txt".


In [11]:
feed_lines_freq

,route_id,route_name,direction_id,window,min_per_trip,ntrips,geometry
0,10232,256 Whitby Estate/Park Royal/Spu...,0,15:00-19:00,30,8,"LINESTRING (-123.1393 49.32689, -123.13777 49...."
1,10232,256 Whitby Estate/Park Royal/Spu...,0,19:00-22:00,45,4,"LINESTRING (-123.1393 49.32689, -123.13777 49...."
2,10232,256 Whitby Estate/Park Royal/Spu...,0,6:00-9:00,36,5,"LINESTRING (-123.1393 49.32689, -123.13777 49...."
3,10232,256 Whitby Estate/Park Royal/Spu...,0,9:00-15:00,30,12,"LINESTRING (-123.1393 49.32689, -123.13777 49...."
4,10232,256 Whitby Estate/Park Royal/Spu...,1,15:00-19:00,30,8,"LINESTRING (-123.16384 49.34953, -123.1632 49...."
...,...,...,...,...,...,...,...
3194,9745,418 Kingswood/22nd St Station ...,1,15:00-19:00,24,10,"LINESTRING (-122.94873 49.1999, -122.94961 49...."
3195,9745,418 Kingswood/22nd St Station ...,1,6:00-9:00,16,11,"LINESTRING (-122.94873 49.1999, -122.94961 49...."
3196,9745,418 Kingswood/22nd St Station ...,1,9:00-15:00,180,2,"LINESTRING (-122.94873 49.1999, -122.94961 49...."
3197,9745,418 Kingswood/22nd St Station ...,1,15:00-19:00,240,1,"LINESTRING (-122.94873 49.1999, -122.94961 49...."


In [151]:
map_kepler = kp.KeplerGl(height=400, data={"Transit Routes": shape_info})
map_kepler.add_data(data=feed_stops, name="Stops")
map_kepler.save_to_html(file_name="gtfs_functions_map.html")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to gtfs_functions_map.html!


In [23]:
route_info = routes[['route_id', 'route_short_name', 'route_long_name', 'route_type', ]].astype('string')
route_info['route_type'] = 'type' + route_info['route_type'] 


line_freq = pd.merge(feed_lines_freq, route_info, on='route_id', how='left')
line_freq
# line_freq

,route_id,route_name,direction_id,window,min_per_trip,ntrips,geometry,route_short_name,route_long_name,route_type
0,10232,256 Whitby Estate/Park Royal/Spu...,0,15:00-19:00,30,8,"LINESTRING (-123.1393 49.32689, -123.13777 49....",<NA>,<NA>,<NA>
1,10232,256 Whitby Estate/Park Royal/Spu...,0,19:00-22:00,45,4,"LINESTRING (-123.1393 49.32689, -123.13777 49....",<NA>,<NA>,<NA>
2,10232,256 Whitby Estate/Park Royal/Spu...,0,6:00-9:00,36,5,"LINESTRING (-123.1393 49.32689, -123.13777 49....",<NA>,<NA>,<NA>
3,10232,256 Whitby Estate/Park Royal/Spu...,0,9:00-15:00,30,12,"LINESTRING (-123.1393 49.32689, -123.13777 49....",<NA>,<NA>,<NA>
4,10232,256 Whitby Estate/Park Royal/Spu...,1,15:00-19:00,30,8,"LINESTRING (-123.16384 49.34953, -123.1632 49....",<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...
3194,9745,418 Kingswood/22nd St Station ...,1,15:00-19:00,24,10,"LINESTRING (-122.94873 49.1999, -122.94961 49....",<NA>,<NA>,<NA>
3195,9745,418 Kingswood/22nd St Station ...,1,6:00-9:00,16,11,"LINESTRING (-122.94873 49.1999, -122.94961 49....",<NA>,<NA>,<NA>
3196,9745,418 Kingswood/22nd St Station ...,1,9:00-15:00,180,2,"LINESTRING (-122.94873 49.1999, -122.94961 49....",<NA>,<NA>,<NA>
3197,9745,418 Kingswood/22nd St Station ...,1,15:00-19:00,240,1,"LINESTRING (-122.94873 49.1999, -122.94961 49....",<NA>,<NA>,<NA>


Jupyter crashed multiple times when trying to load the Folium map. I was just barely able to open it after several tries, with my ram usage exceeding 90%. Based on this, Folium seems like a poor choice for plotting a large amount of points. 

In [ ]:
# map = folium.Map(location=[49.25073221957535, -123.00896884483716], tiles = "CartoDB.Voyager", zoom_start=11, prefer_canvas=True) 
# # Thunderforest.MobileAtlas
# # CartoDB Positron
# # Stadia.AlidadeSmoothDark
# # map

# i = 0
# for coordinates in geo_df_list:
#     map.add_child(
#         folium.Marker(
#             location=coordinates
#         )
#     )
#     i = i + 1

In [25]:
# map_kepler = kp.KeplerGl(height=400, data={"Transit Routes": shapes_with_routes})
# map_kepler.add_data(data=stops, name="Stops")
# map_kepler.save_to_html(file_name="kepler_map.html")

In [ ]:
geometry = gpd.points_from_xy(shapes.shape_pt_lon, shapes.shape_pt_lat)

In [ ]:
geo_df = gpd.GeoDataFrame(
    shapes, geometry=geometry
)

In [ ]:
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]

In [ ]:
# map.save("test.html")

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

In [ ]:
map = px.scatter_geo(shapes_with_routes, lat='shape_pt_lat', lon='shape_pt_lon')

In [ ]:
is_multi = shapes["shape_id"].value_counts() > 1
shapes = shapes[shapes["shape_id"].isin(is_multi[is_multi].index)]

In [43]:
feed_routes['route_id'] = feed_routes['route_id'].astype('int64')

In [47]:
feed_routes[feed_routes['route_id'] == 6770]

,Unnamed: 0,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,route_name
190,190,6770,TL,WCE,West Coast Express ...,,2,,87189d,FFFFFF,WCE West Coast Express ...


In [55]:
feed_trips['route_id'] = feed_trips['route_id'].astype('int64')

In [57]:
feed_trips[feed_trips['route_id'] == 6770]

,trip_id,route_id,pattern_id,route_pattern,pattern_name,route_name,service_id,direction_id,shape_id


In [90]:
route_info[route_info['route_id'] == '6770']

,route_id,route_short_name,route_long_name,route_type
190,6770,WCE,West Coast Express ...,type2


In [110]:
trips[trips['route_id'] == 6770]

,Unnamed: 0,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
50159,50159,6770,155601,14054659,West Coast Express Train To Mission ...,,0,2102991,293180,0,1
50160,50160,6770,155601,14054660,West Coast Express Train To Mission ...,,0,2102990,293180,0,1
50161,50161,6770,155601,14054661,West Coast Express Train To Mission ...,,0,2102989,293180,0,1
50162,50162,6770,155601,14054662,West Coast Express Train To Mission ...,,0,2102988,293180,0,1
50163,50163,6770,155601,14054663,West Coast Express Train To Mission ...,,0,2102987,293180,0,1
50164,50164,6770,155601,14054664,West Coast Express Train To Waterfront ...,,1,2102996,293181,0,1
50165,50165,6770,155601,14054665,West Coast Express Train To Waterfront ...,,1,2102995,293181,0,1
50166,50166,6770,155601,14054666,West Coast Express Train To Waterfront ...,,1,2102994,293181,0,1
50167,50167,6770,155601,14054667,West Coast Express Train To Waterfront ...,,1,2102993,293181,0,1
50168,50168,6770,155601,14054668,West Coast Express Train To Waterfront ...,,1,2102992,293181,0,1


In [120]:
feedTrips['trip_id'] = feedTrips['trip_id'].astype('int64')

In [128]:
route_info

,route_id,route_short_name,route_long_name,route_type
0,10232,256,Whitby Estate/Park Royal/Spuraway ...,type3
1,11201,033,29th Avenue Station/UBC ...,type3
2,11692,364,Langley Centre/Scottsdale ...,type3
3,11693,388,22nd St Station/Carvolth Exchange ...,type3
4,11696,609,South Delta Exchange/Ladner Exchange ...,type3
...,...,...,...,...
232,7322,132,Capitol Hill/Hastings @ Gilmore ...,type3
233,8286,370,Cloverdale/Willowbrook ...,type3
234,8289,865,Samuel Robertson ...,type3
235,8290,341,Guildford/Newton Exchange ...,type3


In [61]:
trip_info = trips[['route_id', 'shape_id']]

In [131]:
trip_route_info = pd.merge(trip_info, route_info, on='route_id', how='left')
shape_info = pd.merge(feed_shapes, trip_route_info, on='shape_id', how='left')
shape_info = shape_info.drop_duplicates()

In [133]:
shape_info

,shape_id,geometry,route_id,route_short_name,route_long_name,route_type
0,292022,"LINESTRING (-123.17228 49.25764, -123.17039 49...",6612,002,Macdonald/Burrard Station ...,type3
83,292023,"LINESTRING (-123.18589 49.23455, -123.18588 49...",6612,002,Macdonald/Burrard Station ...,type3
289,292024,"LINESTRING (-123.12199 49.28646, -123.12086 49...",6612,002,Macdonald/Burrard Station ...,type3
372,292025,"LINESTRING (-123.12199 49.28646, -123.12086 49...",6612,002,Macdonald/Burrard Station ...,type3
578,292026,"LINESTRING (-123.11659 49.20916, -123.1171 49....",6613,003,Main/Waterfront Station ...,type3
...,...,...,...,...,...,...
60688,296953,"LINESTRING (-122.80114 49.03293, -122.80114 49...",31818,362,Seaside/White Rock Centre ...,type3
60779,296954,"LINESTRING (-123.1393 49.32689, -123.13777 49....",38311,R2,Marine Dr ...,type3
60951,298184,"LINESTRING (-123.12341 49.36183, -123.12359 49...",6691,254,British Properties/Park Royal/Van ...,type3
60992,298193,"LINESTRING (-122.80116 49.02758, -122.80254 49...",6705,321,White Rock/Newton/Surrey Central Station ...,type3


In [79]:
route_info['route_id'] = route_info['route_id'].astype('int64')

In [91]:
feed_shapes['shape_id'] = feed_shapes['shape_id'].astype('int64')

In [145]:
shape_info[shape_info['route_long_name'] == "Canada Line"]

,shape_id,geometry,route_id,route_short_name,route_long_name,route_type


In [139]:
shape_info.dtypes

shape_id                     int64
geometry                  geometry
route_id                     int64
route_short_name    string[python]
route_long_name     string[python]
route_type          string[python]
dtype: object

In [147]:
feed_stops

,Unnamed: 0,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,wheelchair_boarding,geometry
0,0,1,50001,Westbound Davie St @ Bidwell St ...,,49.286458,-123.140424,BUS ZN,,0,,1,POINT (-123.14042 49.28646)
1,1,10000,59326,Northbound No. 5 Rd @ McNeely Dr ...,,49.179962,-123.091490,BUS ZN,,0,,1,POINT (-123.09149 49.17996)
2,2,10001,59324,Northbound No. 5 Rd @ Woodhead Rd ...,,49.182670,-123.091448,BUS ZN,,0,,1,POINT (-123.09145 49.18267)
3,3,10002,59323,Southbound No. 5 Rd @ Cambie Rd ...,,49.184252,-123.091627,BUS ZN,,0,,1,POINT (-123.09163 49.18425)
4,4,10003,59325,Southbound No. 5 Rd @ Woodhead Rd ...,,49.182051,-123.091659,BUS ZN,,0,,1,POINT (-123.09166 49.18205)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8784,8784,99954,,Pitt Meadows Station ...,,49.225796,-122.688379,WCE3Z,,1,,1,POINT (-122.68838 49.2258)
8785,8785,99955,,Maple Meadows Station ...,,49.216513,-122.666210,WCE3Z,,1,,1,POINT (-122.66621 49.21651)
8786,8786,99956,,Port Haney Station ...,,49.212202,-122.605240,WCE3Z,,1,,1,POINT (-122.60524 49.2122)
8787,8787,99957,,Mission City Station ...,,49.133694,-122.304898,WCE4Z,,1,,1,POINT (-122.3049 49.13369)


In [11]:
api_key = '5b3ce3597851110001cf6248159f2d494f9c4c5aad2f9e8cfc9fe5bd'

In [13]:
ors_client = ors.Client(key=api_key)